# Introduction to QisKit

# There is nothing to hand out here

First, some libraries to load up front

In [1]:
import numpy as np
from math import pi
try:
    from qiskit import *
    from qiskit_aer import AerSimulator, StatevectorSimulator
    from matplotlib.pyplot import plot,show
except:
    ! python -m pip install matplotlib pylatexenc
    ! python -m pip install qiskit qiskit-aer
from qiskit import *
from qiskit_aer import AerSimulator, StatevectorSimulator
from qiskit.circuit import *
from qiskit.circuit.library import *
from qiskit.quantum_info.operators import Operator
from scipy import optimize
from matplotlib.pyplot import plot,show
%matplotlib inline
%config InlineBackend.figure_format = 'svg' # Makes the images look nice

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 8.4 MB/s eta 0:00:008.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 8.5 MB/s eta 0:00:008.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 8.5 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=275f28beef967baf4d4cdfeb7ccacd93a706cb66b68d5e52ed7cb40ff27817d1
  Stored in directory: /home/benoit/.cache/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 8.2 MB/s eta 0:00:00 MB/s eta 0:00:0101


# 1 - Circuits

A circuit in QisKit acts on quantum *and* classical registers.

A classical register aims at storing the result of the measurement of a quantum register.

In [2]:
# Allocating 2 qubits
q = QuantumRegister(2, name="x")

# Allocating 2 bits
c = ClassicalRegister(2, name="y")

# We build a quantum circuit with both registers.
# By default, everything is initialized to 0 and to |0>
qc = QuantumCircuit(q,c)

# Applying Hadamard on qubit 0:
qc.h(q[0])

# Applying X on qubit 0:
qc.x(q[0])

# Applying z on qubit 0:
qc.z(q[0])

# Applying CNOT on qubits 0 and 1:
qc.cx(q[0],q[1])

# Mesure of all of register q, storing results in c.
# This is still part of the circuit
qc.measure(q, c)

# A summary of native operations can be found here:
# https://qiskit.org/documentation/tutorials/circuits/3_summary_of_quantum_operations.html


The circuit can be drawn in text-style or with mathplotlib. Note how the name given to the registers appear on the drawing. The simple wires are qubit-wires, while the doubled-wires are for bits.

In [3]:
qc.draw()

┌───┐┌───┐┌───┐     ┌─┐   
x_0: ┤ H ├┤ X ├┤ Z ├──■──┤M├───
     └───┘└───┘└───┘┌─┴─┐└╥┘┌─┐
x_1: ───────────────┤ X ├─╫─┤M├
                    └───┘ ║ └╥┘
y: 2/═════════════════════╩══╩═
                          0  1

# 2 - Runing a circuit 

One can run a circuit with **backends**.

Here is a backend emulating the behavior of a quantum co-processor. It makes it possible to *lauch* a series of runs, keeping track of how many of each results were obtained in the classical registers (remember, each run is probabilistic).

In [4]:
q = QuantumRegister(2, name="x")
c = ClassicalRegister(2, name="y") 

qc = QuantumCircuit(q,c)
qc.h(q[0])          # build a 
qc.cx(q[0],q[1])  # Bell state

qc.measure(q, c) # measure of all of q

# To run the circuit, we initialize a simulator
simulator = AerSimulator()

# Then performs several runs of the circuit using this backend. Here we ask for 1024 runs.
job = simulator.run(qc, shots=1024)

# To retrieve the results -- note how we only get values for the single bit-register 
res = dict(job.result().get_counts(qc))

res

{'00': 528, '11': 496}

In what we just saw, we measured **all** of the system. The dictionary stores the number of times each key has been found with the final measure.

We can however measure only part of the system: the rest is "forgotten". For instance, in the following we only measure the 1st qubit

In [5]:
q = QuantumRegister(2, name="x")
c = ClassicalRegister(1, name="y")

qc = QuantumCircuit(q,c)
qc.h(q[0])          # on fabrique un 
qc.cx(q[0],q[1])  # état de Bell


qc.measure(q[0], c[0]) # mesure du premier qubit

simulator = AerSimulator()
job = simulator.run(qc, shots=1024)
res = dict(job.result().get_counts(qc))
res

{'1': 500, '0': 524}

Note how the keys only contain one bit : the content of the classical register 


# 3 - Order of the bits in the keys

Unlike what we saw in class, the bit-vector has to be read "by turning the head on the left" with respect to the circuit: in a register $x$, the qubit $x_0$ (top wire) is the first one. This is also the case for the classical registers.

Below a concrete example:

In [6]:
q = QuantumRegister(2)
c = ClassicalRegister(2)
qc = QuantumCircuit(q,c)

# Applying X on qubit 0
qc.x(q[0])

# So at the end, |x_0 x_1> is in state |10>
qc.measure(q,c)

qc.draw()

# And now, in register c[0] we have 1 and in c[1] we have 0. 
simulator = AerSimulator()
job = simulator.run(qc, shots=1024)
dict(job.result().get_counts(qc))

{'01': 1024}

A key should then be read $b_1b_0$ : the register $c$ is written "$c_1c_0$".

# 4 - Boxing : unitaries and sub-circuits

It is possible to look at a sub-circuit as a unitary gate. This circuit can then be used as many time as needed in another circuit.

Beware: only circuits **without** classical registers can be boxed into a unitary gate...

In [7]:
# Let us build a circuit

q = QuantumRegister(2, name="x")
aux = QuantumCircuit(q) # No classical registers !
aux.h(q[0])
aux.cx(q[0],q[1])

aux.draw()

┌───┐     
x_0: ┤ H ├──■──
     └───┘┌─┴─┐
x_1: ─────┤ X ├
          └───┘

In [8]:
# We can now make a home-made gate using this circuit

o = aux.to_gate(label="moncirc") # name to be used in drawings

In [9]:
# We now have a new gate "o" acting on 2 qubits. We can use it as we want.

q = QuantumRegister(4, name="x")
qc = QuantumCircuit(q)

qc.barrier()  # To horizontally "separate" pieces of circuits, in drawings for instance.
qc.append(o,[q[0],q[1]]) # adding an object "UnitaryGate" can be done with .append
qc.barrier()
qc.append(o,[q[2],q[1]]) # Check the numbering in the drawing !
qc.barrier()
qc.append(o.control(),[q[0],q[2],q[3]]) # We can control a door -- the first wire is the control qubit.

qc.draw()


░ ┌──────────┐ ░              ░             
x_0: ─░─┤0         ├─░──────────────░──────■──────
      ░ │  moncirc │ ░ ┌──────────┐ ░      │      
x_1: ─░─┤1         ├─░─┤1         ├─░──────┼──────
      ░ └──────────┘ ░ │  moncirc │ ░ ┌────┴─────┐
x_2: ─░──────────────░─┤0         ├─░─┤0         ├
      ░              ░ └──────────┘ ░ │  moncirc │
x_3: ─░──────────────░──────────────░─┤1         ├
      ░              ░              ░ └──────────┘

In [10]:
# One can open the boxes

qc.decompose().draw()

# Note how the controlled gate is splitted into native, elementary gates on 1 and 2 qubits.

░ ┌───┐      ░            ░                      
x_0: ─░─┤ H ├──■───░────────────░────────■──────────■──
      ░ └───┘┌─┴─┐ ░      ┌───┐ ░        │          │  
x_1: ─░──────┤ X ├─░──────┤ X ├─░────────┼──────────┼──
      ░      └───┘ ░ ┌───┐└─┬─┘ ░ ┌──────┴───────┐  │  
x_2: ─░────────────░─┤ H ├──■───░─┤ U(π/2,0,π,0) ├──■──
      ░            ░ └───┘      ░ └──────────────┘┌─┴─┐
x_3: ─░────────────░────────────░─────────────────┤ X ├
      ░            ░            ░                 └───┘

# 5 - High-level operations on UnitaryGate

Once we have a UnitaryGate object (for instance the 'o' object), one can perform various operations in it, such as
* power
* control
* inverse

However, the names get all mangled...

Below some examples

In [11]:
q = QuantumRegister(4, name="x")
qc = QuantumCircuit(q)

qc.append(o.power(2),[q[0],q[1]]) # will perform 'o' twice

qc.append(o.power(5).control().control(),[q[2],q[3],q[0],q[1]]) # power and control can be combined 

qc.append(o.inverse(),[q[0],q[1]]) # for the inverse (the _dg in the name stands for "dagger")

qc.append(o.control(num_ctrl_qubits=2, ctrl_state='10'),[q[2],q[3],q[0],q[1]]) 
    # one can perform a bunch of positive and negative controls in one go.
          
qc.draw()

# Ugly names ! But the name somehow keep the power so we know where it comes from.

┌────────────────┐┌────────────────┐┌─────────────────┐┌──────────┐
x_0: ┤0               ├┤0               ├┤0                ├┤0         ├
     │  circuit-170^2 ││  circuit-170^5 ││  circuit-170_dg ││  moncirc │
x_1: ┤1               ├┤1               ├┤1                ├┤1         ├
     └────────────────┘└───────┬────────┘└─────────────────┘└────┬─────┘
x_2: ──────────────────────────■─────────────────────────────────o──────
                               │                                 │      
x_3: ──────────────────────────■─────────────────────────────────■──────

# 6 - Operators

One can ask QisKit to build a circuit from an operator given as a matrix.

In [12]:
# What is doing this gate ?

U = UnitaryGate(
    Operator([[1,0,0,0],
              [0,1,0,0],
              [0,0,1,0],
              [0,0,0,np.exp(pi*1j*1/4)]]), label="MyDoor")

The operator have to be a matrix of size power of $2$ --- the unitary gate then acts on the corresponding number of wires.

In [13]:
q = QuantumRegister(2)
qc = QuantumCircuit(q)

qc.x(q) # State initialized to |11>

qc.append(U,q) # In principle, a phase shift should occur.

qc.draw()

┌───┐┌─────────┐
q6_0: ┤ X ├┤0        ├
      ├───┤│  MyDoor │
q6_1: ┤ X ├┤1        ├
      └───┘└─────────┘

Let us check the phase shift by using another backend: 'statevector_simulator', keeping track of the state vector.

In [14]:
simulator = StatevectorSimulator()
job = simulator.run(qc, memory=True)
job_result = job.result()
job_result.results[0].to_dict()['data']['statevector']

# There has indeed been a phase shift !

Statevector([0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.70710678+0.70710678j],
            dims=(2, 2))
